In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [185]:
df = pd.read_csv('heart.csv')

df = df.drop(['RestingBP', 'RestingECG'], axis=1)

print(df)

     Age Sex ChestPainType  Cholesterol  FastingBS  MaxHR ExerciseAngina  \
0     40   M           ATA          289          0    172              N   
1     49   F           NAP          180          0    156              N   
2     37   M           ATA          283          0     98              N   
3     48   F           ASY          214          0    108              Y   
4     54   M           NAP          195          0    122              N   
..   ...  ..           ...          ...        ...    ...            ...   
913   45   M            TA          264          0    132              N   
914   68   M           ASY          193          1    141              N   
915   57   M           ASY          131          0    115              Y   
916   57   F           ATA          236          0    174              N   
917   38   M           NAP          175          0    173              N   

     Oldpeak ST_Slope  HeartDisease  
0        0.0       Up             0  
1        1.

In [186]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('heart.csv')
df_one_hot_encoded = pd.get_dummies(df)

# Assuming df_one_hot_encoded has features and target variable
X = df_one_hot_encoded.drop(columns='HeartDisease')
y = df_one_hot_encoded['HeartDisease']

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

x_train = x_train.values
y_train = y_train.values
x_test = x_test.values
y_test = y_test.values

print(x_train)

[[57 150 276 ... False True False]
 [62 140 394 ... False True False]
 [51 95 0 ... False True False]
 ...
 [49 130 341 ... False True False]
 [65 150 225 ... False True False]
 [43 150 247 ... False False True]]


In [187]:
# Decision Tree Class
class decision_tree:
    def __init__(self):
        # 결정 트리 노드 속성 초기화
        self.feature = None    # 분할에 사용할 특징 인덱스
        self.value = None      # 분할에 사용할 임계값
        self.left = None       # 왼쪽 서브트리
        self.right = None      # 오른쪽 서브트리
        self.label = None      # 리프 노드일 경우의 클래스 레이블

    def fit(self, x, y):
        # 결정 트리를 재귀적으로 성장시킴
        self.feature, self.value = self.find_best_split(x, y)

        # 만약 최적 분할이 없다면 이 노드는 다수 클래스 레이블로 하는 리프 노드가 됨
        if self.feature is None:
            label, count = np.unique(y, return_counts=True)
            self.label = label[np.argmax(count)]
        else:
            # 데이터를 분할하고 왼쪽과 오른쪽 서브트리를 재귀적으로 성장시킴
            x_left, y_left, x_right, y_right = self.split_data(x, y, self.feature, self.value)
            self.left = decision_tree()
            self.right = decision_tree()
            self.left.fit(x_left, y_left)
            self.right.fit(x_right, y_right)

    def find_best_split(self, x, y):
        # Gini 불순도를 기반으로 최적 분할을 찾음
        parent_gini = self.calculate_gini(y)
        best_gini = parent_gini
        best_feature, best_value = None, None

        # 각 특징 및 해당 값의 고유값을 반복하여 최적 분할을 찾음
        for column in range(x.shape[1]):
            values = np.unique(x[:, column])
            for value in values:
                # 현재 특징과 값에 기반하여 데이터를 분할함
                a, label_left, b, label_right = self.split_data(x, y, column, value)
                gini = self.cost_function(label_left, label_right)

                # 현재 분할이 더 낮은 Gini 불순도를 갖는다면 최적 분할을 업데이트함
                if gini < best_gini:
                    best_gini = gini
                    best_feature, best_value = column, value

        return best_feature, best_value

    def split_data(self, x, y, column, value):
        # 주어진 특징과 값에 기반하여 데이터를 왼쪽과 오른쪽으로 분할함
        left_mask = x[:, column] <= value
        right_mask = x[:, column] > value
        return x[left_mask], y[left_mask], x[right_mask], y[right_mask]

    def calculate_gini(self, y):
        # 레이블 집합에 대한 Gini 불순도를 계산함
        labels, count = np.unique(y, return_counts=True)
        p = count / count.sum()
        gini = 1 - (p ** 2).sum()
        return gini

    def cost_function(self, y_left, y_right):
        # Gini 불순도를 기반으로 분할의 비용을 계산함
        num_total = len(y_left) + len(y_right)
        p_left = len(y_left) / num_total
        p_right = len(y_right) / num_total
        cost = p_left * self.calculate_gini(y_left) + p_right * self.calculate_gini(y_right)
        return cost

    def predict(self, x):
        # 단일 샘플에 대한 예측을 수행함
        if self.label is not None:
            # 리프 노드인 경우 클래스 레이블을 반환함
            return self.label
        else:
            # 트리를 재귀적으로 탐색하여 예측 클래스를 찾음
            if x[self.feature] <= self.value:
                return self.left.predict(x)
            else:
                return self.right.predict(x)

    def prediction(self, x):
        # 다중 샘플에 대한 예측을 수행함
        prediction = np.array([])
        for row in x:
            p = self.predict(row)
            prediction = np.append(prediction, p)
        return prediction

    def accuracy(self, x, y):
        # 주어진 데이터에 대한 모델의 정확도를 계산함
        p = self.prediction(x)
        compare = p == y
        t_f, counts = np.unique(compare, return_counts=True)

        # 정확도를 올바른 예측 수와 전체 예측 수의 비율로 계산함
        acc = counts[np.where(t_f == True)] / len(y)
        return acc


In [196]:
# Decision Tree 모델

tree = decision_tree()

tree.fit(x_train, y_train)
train_acc = tree.accuracy(x_train, y_train)
test_acc = tree.accuracy(x_test, y_test)

print('Decision Tree 결과')
print('train acc :', train_acc[0])
print('test acc :', test_acc[0])

Decision Tree 결과
train acc : 1.0
test acc : 0.8260869565217391


In [204]:
# Random Forest Class
class random_forest:
    def __init__(self, n):
        # 랜덤 포레스트 객체 초기화
        self.n = n
        self.trees = [] # 의사결정 트리를 저장할 리스트

    def bagging(self, x_train, y_train):
        # Bagging을 수행하여 랜덤 포레스트 훈련
        for i in range(self.n):
            index = len(x_train)
            random_index = np.random.choice(index, index, True)
            random_x_train = x_train[random_index]
            random_y_train = y_train[random_index]

            tree = decision_tree()  # 의사결정 트리 객체 생성
            tree.fit(random_x_train, random_y_train)  # 의사결정 트리 훈련
            self.trees.append(tree)

    def rf_predict(self, x_test):
        # 랜덤 포레스트로 예측 결과 결합
        predictions = []
        for tree in self.trees:
            p = tree.prediction(x_test)
            predictions.append(p)

        # 각 트리의 예측 결과를 평균하여 최종 예측 결과 계산
        prediction = np.mean(predictions, axis=0)
        return np.round(prediction)  # 이진 분류를 가정하고 있으므로 반올림하여 0 또는 1로 변환

    def rf_accuracy(self, x_test, y_test):
        # 랜덤 포레스트의 정확도 계산
        p = self.rf_predict(x_test)
        compare = p == y_test
        t_f, counts = np.unique(compare, return_counts=True)
        acc = counts[np.where(t_f == True)] / len(y_test)
        return acc

In [210]:
# Random Forest 모델

forest = random_forest(10)

forest.bagging(x_train, y_train)
train_acc = forest.rf_accuracy(x_train, y_train)
test_acc = forest.rf_accuracy(x_test, y_test)

print('Random Forest 결과')
print('train acc :', train_acc[0])
print('test acc :', test_acc[0])

Random Forest 결과
train acc : 0.9918256130790191
test acc : 0.9021739130434783


In [252]:
class GradientBoosting:
    def __init__(self, n_estimators=10, learning_rate=0.1):
        # GradientBoosting 객체 초기화
        '''
        n_estimators : 사용할 의사결정 트리의 개수
        learning_rate : 학습률
        '''
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.pred = 0
        self.trees = []

    def fit(self, X, y):
        # 초기 예측값은 로지스틱 함수 적용
        initial_prediction = np.log(np.mean(y) / (1 - np.mean(y)))
        prediction = initial_prediction
        self.pred = np.log(np.mean(y))

        for i in range(self.n_estimators):
            # pseudo residual 계산
            residual = y - np.exp(prediction) / (1 + np.exp(prediction))

            # 의사결정 트리 생성
            tree = decision_tree()
            tree.fit(X, residual)

            prediction += self.learning_rate * tree.prediction(X)

            self.trees.append(tree)

    def predict(self, X):
        prediction = self.pred

        # 각 트리의 예측 결과를 학습률과 곱하여 현재 예측값에 더함
        for tree in self.trees:
            prediction += self.learning_rate * tree.prediction(X)

        # x>0 ? 1 : 0 으로 이진 분류
        return (prediction > 0).astype(int)

    def accuracy(self, X, y):
        p = self.predict(X)
        acc = np.mean(p == y)
        return acc

In [253]:
# Gradient Boosting 모델

GBM = GradientBoosting(n_estimators=10, learning_rate=0.5)

GBM.fit(x_train, y_train)
train_acc = GBM.accuracy(x_train, y_train)
test_acc = GBM.accuracy(x_test, y_test)

print('Gradient Boosting 결과')
print('train acc :', train_acc)
print('test acc :', test_acc)

Gradient Boosting 결과
train acc : 1.0
test acc : 0.8260869565217391
